# Cricket Simulator
1. Database search & get data
2. EDA
3. EDA Plots
4. Simulator for scores

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Global Variables & functions

In [3]:
p_names = {
    'ind': {
        'bt': ['rsharma', 'klrahul', 'vkohli', 'arahane', 'stendulkar', 'rdravid', 'sganguly'], 
        'wk': ['dhoni'], 'all': ['hpandya', 'jadeja', 'kpandya', 'ysingh'], 
        'bl': ['bkumar', 'rashwin', 'mdshami', 'bumrah', 'akumble', 'hsingh', 'isharma', 'zkhan', 'anehra']
    }, 
           
    'aus': {
        'bt': ['mhayden', 'rponting', 'mhussey', 'jlanger', 'ssmith'], 
        'wk': ['agilchrist'], 'all': ['symonds', 'abitchell'], 
        'bl': ['swarne', 'mmarsh', 'blee', 'gmcgrath']
    }, 
    
    'saf': {
        'bt': ['gkirsten', 'bdipenner', 'abdevilliers', 'hgibbs', 'faulkner', 'gsmith'], 
        'wk': ['mboucher'], 'all': ['spollock', 'kallis'], 
        'bl': ['adonald', 'mntini', 'dsteyn']
    }, 
    
    'win': {
        'bt': ['cgayle', 'whinds', 'blara', 'schanderpaul'], 
        'wk': ['keepr'], 'all': ['pollard', 'bravo'], 
        'bl': ['bkumar', 'rashwin', 'mdshami', 'bumrah']
    }, 
    
    'sri': {
        'bt': ['kaluwithrana', 'atapattu', 'jayavardene'], 
        'wk': ['sangakara'], 'all': ['jayasuriya', 'adesilva'], 
        'bl': ['cvaas', 'murali', 'fernando', 'yokumar']
    }
          }

In [38]:
def reset_wkts_proba(p_skill = 'bl'):
    temp_proba = {
        0: 0.96, 1: 0.04} if p_skill=='bl' else {
        0: 0.991, 1: 0.009} if p_skill=='all' else {
        0: 0.9991, 1: 0.0009} if p_skill == 'bt' else {
        0: 0.9999, 1: 0.0001}
    dict_sum = pd.Series(list(temp_proba.values())).sum()
    return {k: v/dict_sum for k, v in list(temp_proba.items())}  # normalize so sum of proba = 1

In [43]:
def reset_runs_proba(p_skill = 'bt'):
    temp_proba = {
    0: 0.55, 1: 0.25, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.05} if p_skill=='bt' else {
    0: 0.65, 1: 0.22, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.10} if p_skill=='all' else {
    0: 0.70, 1: 0.20, 2: 0.1, 3: 0.005, 4: 0.15, 5: 0.001, 6: 0.15} if p_skill == 'wk' else {
    0: 0.80, 1: 0.1, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.15}
    dict_sum = pd.Series(list(temp_proba.values())).sum()
    return {k: v/dict_sum for k, v in list(temp_proba.items())}  # normalize so sum of proba = 1

# Class Definitions

### Team Class

In [6]:
class Team():
    '''
    Class of Team object
    Attributes like team name, matches played, wins, etc. 
    will be part of this class
    '''
    
    def_compos = {'bt': 4, 'wk': 1, 'all': 2, 'bl': 4}
    
    def __init__(self, team_name):
        self.team_name = team_name
        self.m_played = 0
        self.m_wins = 0
        self.team_compos = Team.def_compos
        self.players = self.generate_team()
        
        
    def generate_team(self):
        curr_team = {skill: list(np.random.choice(p_names[self.team_name][skill], self.team_compos[skill]))
                     for skill in ['bt', 'wk', 'all', 'bl']}
        self.players_obj_list = {curr_team[sk][i]: Player(p_name = curr_team[sk][i], p_skill = sk) for sk in ['bt', 'wk', 'all', 'bl'] for i in range(len(curr_team[sk]))}      
        return curr_team
    
    
    def update_team_comp(self, sk_bt = None, sk_wk = None, sk_all = None, sk_bl = None):
        
        if sk_bt != None:
            sk_bt = self.team_compos['bt']
        if sk_wk != None:
            sk_wk = self.team_compos['wk']
        if sk_all != None:
            sk_all = self.team_compos['all']
        if sk_bl != None:
            sk_bl = self.team_compos['bl']
        
        # check for sum = 11
        pl_in_team = sk_bt + sk_wk + sk_all + sk_bl
        if pl_in_team != 11:
            print(f"Count of players NOT equal to 11")
        else:
            self.team_compos['bt'] = sk_bt
            self.team_compos['wk'] = sk_wk
            self.team_compos['all'] = sk_all
            self.team_compos['bl'] = sk_bl
            
            # regenerate team with new comp only if update is correct
            self.players = self.generate_team()  
    

### Player Class 

In [8]:
class Player():
    '''
    Class of Player object
    Attributes like player name, matches played, runs, wickets, wins, etc. 
    will be part of this class
    '''
    
#     wkts_proba = reset_wkts_proba()
#     runs_proba = reset_runs_proba()
    
    def __init__(self, p_name, p_skill):
        self.p_name = p_name
        self.p_skill = p_skill  # batsman, bowler, wk
        self.m_played = 0
        self.bt_runs = 0
        self.bt_balls = 0
        self.bl_wkts = 0
        self.bl_runs = 0
        self.bl_balls = 0
        self.m_wins = 0
        self.runs_proba = reset_runs_proba(p_skill)
        self.wkts_proba = reset_wkts_proba(p_skill)
    
    def updateRunProba(self, run_in_ball):
        '''
        probability at which player scores runs
        default: reset_runs_proba(p_skill)
        '''
        pass
        
        
    def update_bt_runs(self):
        pass
    
    
    def update_bt_balls(self):
        pass
    
    
    def update_bl_wkts(self):
        pass
    
    
    def update_bl_runs(self):
        pass
    
    
    def update_bl_balls(self):
        pass
    
    
    def update_score(self):
        pass

### Match Class

In [9]:
class Match():
    '''
    Class of Match object
    Attributes like teams playing, runs scored, wickets taken, etc. 
    will be part of this class
    '''
    match_cnt = 0
    match_dict = {}
    
    def __init__(self, team1, team2):
        Match.match_cnt += 1
        Match.match_dict[Match.match_cnt] = [team1, team2]
        self.teams = [team1, team2]
        self.first_bt_team = np.random.choice(self.teams, 1)[0]
        self.second_bt_team = list(set(self.teams).difference(set(self.bt_team)))[0]
        

### Innings Class

In [10]:
class Innings(Match):
    '''
    Class of Innings object
    Attributes like runs scored, wickets taken, etc. 
    will be part of this class
    '''
    
    wkts_proba = {0: 995, 1: 0.005}
    runs_proba = {0: 0.6, 1: 0.23, 2: 0.1, 3: 0.005, 4: 0.054, 5: 0.001, 6: 0.01}
    
    def __init__(self, team1, team2):
        
        self.bt_team = bt_team
        self.bl_team = bl_team
        # batting stats
        self.runs = 0
        self.fours = 0
        self.sixes = 0
        self.boundary = 0
        # bowling stats
        self.wkts = 0
        self.wide = 0
        self.noball = 0
        self.bye = 0
        self.lbye = 0
        
    
    def simulate_ball(self, legal_flag = True):
        if legal_flag:
            balls += 1
            runs += Match.simulate_run()
            if (runs == 4) or (runs == 6):
                self.boundary += 1
                if runs == 4:
                    self.fours += 1
                else: 
                    self.sixes += 1
            
            
    # @ class method
    def simulate_run():
        run_in_ball = np.random.choice(a = [0, 1, 2, 3, 4, 5, 6], p = pd.Series(list(runs_proba.values())))
        return run_in_ball

# Initialize Teams

In [11]:
team_name = ['ind', 'saf', 'win', 'sri', 'aus']
team_list = {tn: Team(tn) for tn in team_name}

In [12]:
TEAMNAME = 'ind'
team_list[TEAMNAME].team_name
team_list[TEAMNAME].players

'ind'

{'bt': ['rdravid', 'klrahul', 'arahane', 'klrahul'],
 'wk': ['dhoni'],
 'all': ['kpandya', 'jadeja'],
 'bl': ['bumrah', 'anehra', 'zkhan', 'hsingh']}

In [13]:
team_list[TEAMNAME].players_obj_list.keys()

dict_keys(['rdravid', 'klrahul', 'arahane', 'dhoni', 'kpandya', 'jadeja', 'bumrah', 'anehra', 'zkhan', 'hsingh'])

In [14]:
PLNAME = list(team_list[TEAMNAME].players_obj_list.keys())[3]
team_list[TEAMNAME].players_obj_list[PLNAME].p_name
team_list[TEAMNAME].players_obj_list[PLNAME].bt_runs

'dhoni'

0

In [15]:
team_list.keys()

dict_keys(['ind', 'saf', 'win', 'sri', 'aus'])

# Test Simulation

In [19]:
def simulate_score(wkts, balls, runs_proba, wkts_proba):
    
    balls += 1
    wkts_in_ball = np.random.choice(a = list(wkts_proba.keys()), p = pd.Series(list(wkts_proba.values())))
    if wkts_in_ball:
        wkts += 1
        run_in_ball = -1
        runs_proba = reset_runs_proba()
    else: 
        run_in_ball = np.random.choice(a = list(runs_proba.keys()), p = pd.Series(list(runs_proba.values())))
        
        # runs proba updated as per run scored in prev ball
        for k, v in list(runs_proba.items()):
            if k == run_in_ball:
                if run_in_ball == 0:
                    runs_proba[k] = v*1.05
                else:
                    runs_proba[k] = v*1.2
            elif k > run_in_ball:
                runs_proba[k] = v*1.01
            else:
                runs_proba[k] = v
        
        dict_sum = pd.Series(list(runs_proba.values())).sum()
        runs_proba = {k: v/dict_sum for k, v in list(runs_proba.items())}  # normalize so sum of proba = 1
    
    return (wkts, wkts_in_ball, run_in_ball, balls, runs_proba, wkts_proba)

In [20]:
def run_bkts(ball_run_dict):
    temp_df = pd.Series(list(ball_run_dict.values())).to_frame()
    temp_df.columns = ['runs']

    score_df = temp_df['runs'].value_counts().to_frame()
    score_df['score'] = score_df.index
    score_df['run_sum'] = score_df['score'] * score_df['runs']

    return score_df

In [21]:
def innings_simulator():
    wkts = 0
    balls = 0
    runs = 0
    ball_run_dict = {}
    ball_wkt_dict = {}

    wkts_proba = reset_wkts_proba()
    runs_proba = reset_runs_proba()
    
    while wkts <= 10: 
        if balls >= 300:
            break
        else:
            wkts, wkts_in_ball, run_in_ball, balls, runs_proba, wkts_proba = simulate_score(wkts, balls, runs_proba, wkts_proba)
            ball_run_dict[balls] = run_in_ball
            ball_wkt_dict[balls] = wkts_in_ball
    #         f"balls: {balls} | wkts: {wkts} | run: {run_in_ball}"
            runs += run_in_ball if run_in_ball != -1 else 0
    print(f"overall runs: {runs}")
    print(f"balls: {balls}")
    print(f"wkts: {pd.Series(list(ball_wkt_dict.values())).sum()}")
    
    return run_bkts(ball_run_dict), ball_run_dict

In [25]:
score_df, ball_run_dict = innings_simulator()
score_df

overall runs: 339
balls: 300
wkts: 9


,runs,score,run_sum
0,148,0,0
1,82,1,82
4,27,4,108
6,20,6,120
2,13,2,26
-1,9,-1,-9
3,1,3,3


In [ ]:
runs_proba = reset_runs_proba()
runs_proba

In [ ]:
f"{'-'*5} Initial prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}
run_in_ball = np.random.choice(a = list(runs_proba.keys()), p = pd.Series(list(runs_proba.values())))
run_in_ball
runs_proba = {k: v*1.05 if run_in_ball == 0 else v*1.3 if run_in_ball<=k else v for k, v in runs_proba.items()}
f"{'-'*5} After update prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}
dict_sum = pd.Series(list(runs_proba.values())).sum()
runs_proba = {k: v/dict_sum for k, v in list(runs_proba.items())}  # normalize so sum of proba = 1
f"{'-'*5} Normalized prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}